***필요한 라이브러리 설치***

In [8]:
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
# !pip3 install mecab-python3
# !pip3 install konlpy
# !pip install gensim
# !conda install -c conda-forge scikit-learn --yes

***'여성 숏패딩' 카테고리 csv 파일 하나 가져오기***

In [73]:
import pandas as pd
df = pd.read_csv('./cat_id_310090060_DF.csv')
df.head()

,id,name,cat_id,image_url,low_price
0,40007698241,코튼 나일론 스트링 숏 아우터 8SECONDS 323339LY40,310090060,https://shopping-phinf.pstatic.net/main_400076...,35912
1,38543761083,온앤온 NC11 니트 카라 덕 다운 숏 패딩 NJP2WMA02,310090060,https://shopping-phinf.pstatic.net/main_385437...,148230
2,35692762113,폭스퍼 슬림 숏 덕다운 A224PWDA07,310090060,https://shopping-phinf.pstatic.net/main_356927...,125000
3,39838075591,컴포트핏 집업 숏 아우터 8SECONDS 323239LYC5,310090060,https://shopping-phinf.pstatic.net/main_398380...,31913
4,82785708214,[당일출고]오리털 여성 경량패딩 슬림 이너패딩,310090060,https://shopping-phinf.pstatic.net/main_827857...,22800


***Word2Vec, FastText, TF-IDF 활용***

- 최대한 숏패딩과 가까운 데이터를 가져옴으로서
- 전처리 혹은 레이블링 작업 과정 수요를 더 수월하게 하기 위해서 

In [74]:
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
from konlpy.tag import Mecab
import gensim

**토크 나이징**

In [75]:
# 형태 분석기 Okt, Mecab, Komoran, Hannanum, Kkma 트라이 해보기
tokenizer = Mecab()

tokens = []
for idx in range(len(df)):
    token = tokenizer.morphs(df.loc[idx]['name'])
    if len(token) > 1:
        tokens.append(token)

In [95]:
# TF-IDF 용 리스트
docs = list(df['name'])

**Word2Vec 학습**

파라미터
- sg: 0 (CBOW), 1(skip-gram)
- vector_size : 벡터의 차원수
- window : 현재 단어와 예측 단어 간의 거리
- alpha : 초기 학습률
- seed : 임의 숫자를 생성할 때 사용하는 값
- min_count : 빈도수가 이 값보다 낮으면 그 단어를 제거 (빈도가 적은 단어들은 학습하지 않는다)
- workers : 학습을 위한 프로세스 수

In [78]:
model1 = Word2Vec(tokens, vector_size=100, window=5, min_count=5, workers=4, sg=0)

model1.wv.most_similar('패딩', topn=30)


[('조끼', 0.7213610410690308),
 ('레이어드', 0.6834876537322998),
 ('베스트', 0.682614803314209),
 ('초겨울', 0.6800909638404846),
 ('가벼운', 0.6774418950080872),
 ('오리털', 0.6633782386779785),
 ('골덴', 0.6629576683044434),
 ('스커트', 0.6586331725120544),
 ('반', 0.6576777696609497),
 ('버튼', 0.6495097875595093),
 ('수입', 0.6481094360351562),
 ('빈티지', 0.6475752592086792),
 ('레드', 0.6439412832260132),
 ('잠바', 0.6435971260070801),
 ('하이넥', 0.6399156451225281),
 ('학생', 0.6387754082679749),
 ('여친', 0.6386480331420898),
 ('금장', 0.6349058747291565),
 ('유광', 0.6346677541732788),
 ('모자', 0.6295254826545715),
 ('점퍼', 0.6289642453193665),
 ('체크', 0.6265172362327576),
 ('이너', 0.6263688802719116),
 ('박시', 0.619596004486084),
 ('항공', 0.6190775632858276),
 ('퀼팅', 0.6179395318031311),
 ('다이아', 0.6178331971168518),
 ('절기', 0.6164189577102661),
 ('베이직', 0.6161524057388306),
 ('스트링', 0.6146500110626221)]

In [79]:
for _ in range(30):
    model1 = Word2Vec(vector_size=100, min_count=5, window=5, workers=4, sg=0)
    model1.build_vocab(tokens)
    total_examples = model1.corpus_count
    model1.train(tokens, total_examples=model1.corpus_count, epochs=10)

In [84]:
results = model1.wv.most_similar(positive=['숏', '패딩'], topn=20)
keywords_list = [result[0] for result in results]
keywords = str('|'.join(keywords_list))
print(keywords)
results

롱|레이어드|조끼|오리털|하프|퀄팅|체크|누빔|블랙|경량|잠바|골덴|가벼운|다이아|단추|파스텔|깔깔이|꽃무늬|유광|베스트


[('롱', 0.5765172243118286),
 ('레이어드', 0.5576663613319397),
 ('조끼', 0.5551348328590393),
 ('오리털', 0.549066960811615),
 ('하프', 0.5466918349266052),
 ('퀄팅', 0.5444952845573425),
 ('체크', 0.5442023277282715),
 ('누빔', 0.5370717644691467),
 ('블랙', 0.532168984413147),
 ('경량', 0.5258167386054993),
 ('잠바', 0.514199435710907),
 ('골덴', 0.5093244910240173),
 ('가벼운', 0.5066898465156555),
 ('다이아', 0.5041057467460632),
 ('단추', 0.49633634090423584),
 ('파스텔', 0.4796222448348999),
 ('깔깔이', 0.47543689608573914),
 ('꽃무늬', 0.4744662940502167),
 ('유광', 0.4721851646900177),
 ('베스트', 0.47175562381744385)]

In [234]:
del model1

In [86]:
short_jumper_df = df[~df['name'].str.contains('롱|바람막이')].copy()
short_jumper_df['name'][short_jumper_df['name'].str.contains(keywords)]

4                                [당일출고]오리털 여성 경량패딩 슬림 이너패딩
5                    숏패딩 여성 경량 패딩 오리털 퀼팅점퍼 덕다운 다이아 유니클로 대체
14                               시슬리 구스다운 경량 숏패딩 SAPDC2161
18                       블루페페 세이브존06 구스 경량 숏 패딩 P214PSJ841
35                                       여성 노카라 경량 숏패딩 깔깔이
                               ...                        
11989    나이키 NSW 써마핏 시티 시리즈 재킷 덕다운 숏패딩 골프 경량 DH4080-601  M
11990    나이키 NSW 써마핏 시티 시리즈 재킷 덕다운 숏패딩 골프 경량 DH4080-010  L
11992     숏타입 경량 ERKE 패딩 야외운동 스타일 여성겨울자켓  C02- 카키  T07-4XL
11998                        아디다스 우먼스 블랙 후드 숏 패딩 H18637  M
11999       나이키 여성 숏패딩 Air Synthetic 화이트 블랙 DH1367-113  XL
Name: name, Length: 5149, dtype: object

**FastText 학습**

In [88]:
model2 = FastText(tokens, vector_size=100, window=5, min_count=5, workers=4, sg=0)
model2.wv.most_similar(['숏', '패딩'], topn=30)

[('초경량', 0.8560408353805542),
 ('퀄팅패딩', 0.8465202450752258),
 ('레이어드', 0.8432528972625732),
 ('조끼', 0.8343632817268372),
 ('경량', 0.8300626277923584),
 ('모자', 0.827700138092041),
 ('크롭', 0.8234612941741943),
 ('파스텔', 0.820763111114502),
 ('수입', 0.8203874826431274),
 ('잠바', 0.8155559301376343),
 ('초겨울', 0.8141323328018188),
 ('유광', 0.8111475706100464),
 ('여학생', 0.8028049468994141),
 ('오리', 0.7995994687080383),
 ('골덴', 0.798895537853241),
 ('중학생', 0.7966083884239197),
 ('오리털', 0.7958369851112366),
 ('버튼', 0.7913188934326172),
 ('점퍼', 0.789621889591217),
 ('외출', 0.7891978025436401),
 ('베스트', 0.7890758514404297),
 ('가벼운', 0.788623571395874),
 ('만', 0.7882905006408691),
 ('단추', 0.7872329354286194),
 ('학생', 0.7844907641410828),
 ('베이직', 0.7841367721557617),
 ('퀼팅', 0.7838664054870605),
 ('라운드', 0.7798560261726379),
 ('데이트', 0.7785240411758423),
 ('금장', 0.7771201133728027)]

In [ ]:
model2.wv.

**TF-IDF 학습**

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

vectorizer = TfidfVectorizer(max_features=5000)
tf_idf_vector = vectorizer.fit_transform(docs)

In [110]:
tf_idf_vector.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [112]:
mecab = Mecab()
mecab.morphs('숏패딩')

['숏', '패딩']